In [39]:
train_path = r'/Users/zheyuma/MZY/Tsinghua/课程/入侵检测/恶意流量检测/train_data'
test_path = r'/Users/zheyuma/MZY/Tsinghua/课程/入侵检测/恶意流量检测/test_data'
train_feature_path = r'/Users/zheyuma/MZY/Tsinghua/课程/入侵检测/恶意流量检测/train_feature.csv'
test_feature_path = r'/Users/zheyuma/MZY/Tsinghua/课程/入侵检测/恶意流量检测/test_feature.csv'
black_list_file = r'/Users/zheyuma/Desktop/datacon/Datacon2020/恶意流量/phase2_label/blackIPlist.txt'
white_list_file = r'/Users/zheyuma/Desktop/datacon/Datacon2020/恶意流量/phase2_label/whiteIPlist.txt'

In [11]:
black_list = []
white_list = []

In [12]:
with open(black_list_file, 'r') as f:
    lines = f.readlines()
    for line in lines:
        black_list.append(line.strip())
with open(white_list_file, 'r') as f:
    lines = f.readlines()
    for line in lines:
        white_list.append(line.strip())

In [ ]:
black_list

In [25]:
import os
files = os.listdir(test_path)

In [ ]:
import json
import csv
for file in files:
    path = os.path.join(test_path, file)
    print("Processing " + path)
    host = file[:-5]
    if host in black_list:
        label = 1
    else:
        label = 0
    cmd = f'tshark -T json -r {path} -e frame.len -e ip.src -e ip.dst'
    result = os.popen(cmd)
    result = str(result.read())
    json_result = json.loads(result)
    count = 0
    probability = {}
    for i in range(101):
        probability[(i, 0)] = probability[(i, 1)] = 0
    for item in json_result:
        count += 1
        layers = item.get("_source").get("layers")
        length = int(layers.get("frame.len")[0])
        length = int(length / 15)
        if not layers.__contains__("ip.src") or not layers.__contains__("ip.dst"):
            break
        ip_src = layers.get("ip.src")[0]
        ip_dst = layers.get("ip.dst")[0]
        if ip_src == host:
            direction = 0
        else:
            direction = 1
        probability[(length, direction)] += 1
    for key, value in probability.items():
        probability[key] = value / count
    feature = []
    feature.append(host)
    feature.append(label)
    for value in probability.values():
        feature.append(value)
    with open(test_feature, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(feature)

In [ ]:
class FCNet(nn.Module):
    def __init__(self):
        super(FCNet, self).__init__()
        self.fc_layer_1 = nn.Linear(202, 512)
        self.fc_layer_2 = nn.Linear(512, 32)
        self.fc_layer_3 = nn.Linear(32, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.fc_layer_1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc_layer_2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc_layer_3(x)
        return x

class CNN(nn.Module):
    def sub_block(self, in_channels, out_channels, kernel_size=3):
        block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        return block
    
    def __init__(self):
        super(CNN, self).__init__()
        self.convnet1 = self.sub_block(1, 8)
        self.convnet2 = self.sub_block(8, 16)
        self.fc_layer1 = nn.Linear(400, 128)
        self.fc_layer2 = nn.Linear(128, 50)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.convnet1(x)
        x = self.convnet2(x)
        x = torch.flatten(x, 1)
        # print(x.size())
        # exit()
        x = self.fc_layer1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc_layer2(x)
        return x

In [ ]:
def NetworkClassfy(X, Y, X_test, Y_test, X_test_IP):
    black_file = globalValue.answer_black_file
    white_file = globalValue.answer_white_file
    black_list = []
    white_list = []
    with open(black_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            black_list.append(line)
    with open(white_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            white_list.append(line)
    model = FCNet()
    # model = CNN()
    loss_criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_list = []
    acc_list = []
    X = torch.from_numpy(X.values.astype(np.float32))
    X_test = torch.from_numpy(X_test.values.astype(np.float32))
    # Y = torch.from_numpy(Y.values.astype(np.float32))
    # print(X, Y)
    # exit()
    epochs = 50

    for epoch in range(epochs):
        total = 0
        correct = 0
        train_loss = 0
        model.train()
        length = len(X)
        for i in range(length):
            if Y[i] == 'black':
                label = torch.LongTensor([1])
            else:
                label = torch.LongTensor([0])
            
            output = model(X[i].reshape(-1, 202))
            # print(output.data, label)
            # print(output.data.shape, label.shape)
            loss = loss_criterion(output, label)
            predict = torch.max(output.data, 1)[1]
            total += 1
            correct += (predict == label).sum()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        avg_loss = train_loss/total
        loss_list.append(avg_loss)
    
        predict_black_list = []
        predict_white_list = []
        model.eval()
        with torch.no_grad():
            total = 0
            correct = 0
            length = len(X_test)
            for i in range(length):
                output = model(X[i].reshape(-1, 202))
                predict = torch.max(output.data, 1)[1]
                total += 1
                # correct += (predict == label).sum()
                # print(predict)
                if predict == torch.LongTensor([1]):
                    predict_black_list.append(X_test_IP[i] + '\n')
                else:
                    predict_white_list.append(X_test_IP[i] + '\n')
                if predict == torch.LongTensor([1]) and X_test_IP[i] in black_list or \
                    predict == torch.LongTensor([0]) and X_test_IP[i] in white_list:
                    correct += 1
        avg_acc = correct / total
        acc_list.append(avg_acc)
        print ('Epoch [{}/{}], Loss: {:.6f}, Acc: {:.6f}' .format(epoch+1, epochs, avg_loss, avg_acc))
        len_predict_white_file = globalValue.len_predict_white_file
        len_predict_black_file = globalValue.len_predict_black_file
        with open(len_predict_black_file, "w") as f:
            f.writelines(predict_black_list)
        with open(len_predict_white_file, "w") as f:
            f.writelines(predict_white_list)

In [7]:
import pandas as pd
train_feature = pd.read_csv(train_feature_path, header=None)
test_feature = pd.read_csv(test_feature_path, header=None)

In [47]:
train_feature[:]

,0,1,2,3,4,5,6,7,8,9,...,194,195,196,197,198,199,200,201,202,203
0,192.168.159.4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.312500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.062500
1,192.168.212.230,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.201493,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.134328
2,192.168.232.107,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.239130,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.072464
3,192.168.61.78,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.139535,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.255814
4,192.168.208.148,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.363636,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.045455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,192.168.182.38,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.086207,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.465517
5996,192.168.29.134,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.000000
5997,192.168.202.91,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.247934,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.181818
5998,192.168.159.96,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.269058,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.130045


In [8]:
train_data = train_feature.loc[:, 2:]
train_label = train_feature.loc[:, 1]

In [17]:
test_data = test_feature.loc[:, 2:]
test_label = test_feature.loc[:, 1]
test_IP = test_feature.loc[:, 0]

In [10]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(train_data, train_label)

RandomForestClassifier()

In [14]:
results = rfc.predict(test_data)

In [15]:
results

array([0, 0, 1, ..., 0, 1, 0])

In [23]:
white_count = 0
black_count = 0
count = 0
result_black_list = []
result_white_list = []
for result in results:
    if result == 0:
        white_count += 1
        result_white_list.append(test_IP[count] + '\n')
    else:
        black_count += 1
        #print(test_IP[count], end=' ')
        result_black_list.append(test_IP[count] + '\n')
    count += 1

In [24]:
black_count

1915

In [17]:
import os
import json
white_pcap_path = r'/Users/zheyuma/Desktop/datacon/Datacon2020/恶意流量/white/'
black_pcap_path = r'/Users/zheyuma/Desktop/datacon/Datacon2020/恶意流量/black/'
train_csv_path = r'/Users/zheyuma/Desktop/datacon/Datacon2020/恶意流量/train.csv'
test_csv_path = r'/Users/zheyuma/Desktop/datacon/Datacon2020/恶意流量/test.csv'

In [3]:
files_white = os.listdir(white_pcap_path)
files_black = os.listdir(black_pcap_path)

In [9]:
index_path = r'/Users/zheyuma/Desktop/datacon/Datacon2020/恶意流量/index.txt'

In [11]:
index = {}

In [37]:
def add_index(file):
    cmd = f'tshark -r {file} -T json  -e x509sat.printableString'
    result = os.popen(cmd)
    result = str(result.buffer.read().decode(encoding='utf8'))
    json_result = json.loads(result)
    subject = []
    for item in json_result:
        x509sat = item.get('_source').get('layers')
        if not x509sat:
            continue
        subject.extend(x509sat.get('x509sat.printableString'))
    for item in subject:
        if item.lower() in index.keys():
            continue
        index[item.lower()] = 0

In [38]:
for file in files_white:
    path = os.path.join(white_pcap_path, file)
    add_index(path)

In [41]:
for file in files_black:
    path = os.path.join(black_pcap_path, file)
    add_index(path)

In [109]:
with open(index_path, 'r') as f:
    lines = f.readlines()
    for key in lines:
        index[key.strip('\n')] = 0

In [112]:
bagging = index.copy()

In [114]:
bagging['baidu.com'] += 1

In [ ]:
index

In [44]:
with open(index_path, 'w') as f:
    for key in index.keys():
        f.write(key + '\n')

In [117]:
def analyze(file, train=True):
    cmd = f'tshark -r {file} -T json  -e x509sat.printableString'
    result = os.popen(cmd)
    result = str(result.buffer.read().decode(encoding='utf8'))
    json_result = json.loads(result)
    subject = []
    for item in json_result:
        x509sat = item.get('_source').get('layers')
        if not x509sat:
            continue
        subject.extend(x509sat.get('x509sat.printableString'))
    bagging = index.copy()
    for item in subject:
        key = item.strip('\n').lower()
        if key not in bagging:
            continue
        bagging[key] += 1
    df = pd.DataFrame(bagging, index=[0])
    df = pd.concat([pd.DataFrame({'file': file}, index=[0]), df], axis=1)
    if train:
        csv_path = train_csv_path
    else:
        csv_path = test_csv_path
    if os.path.exists(csv_path):
        df.to_csv(csv_path, index=False, header=False, mode='a', sep=',')
    else:
        df.to_csv(csv_path, index=False, header=df.keys(), mode='a', sep=',')

In [118]:
import pandas as pd
for file in files_white:
    path = os.path.join(white_pcap_path, file)
    analyze(path)

In [119]:
for file in files_black:
    path = os.path.join(black_pcap_path, file)
    analyze(path)

In [120]:
train_data = pd.read_csv(train_csv_path)

In [121]:
train_data = train_data.drop(['file'], axis=1)

In [122]:
train_label = []

In [123]:
for i in os.listdir(white_pcap_path):
    train_label.append(0)

In [124]:
for i in os.listdir(black_pcap_path):
    train_label.append(1)

In [125]:
train_data

,globalsign nv-sa,globalsign rsa ov ssl ca 2018,massachusetts,boston,"logmein, inc.",secure.logmeinrescue.com,globalsign root ca - r3,globalsign,globalsign organization validation ca - sha256 - g2,beijing,...,bigoterr3a.com,"block.io, inc.",block.io,it department,lct.avictc.com,vxcharts.com,bdns.at,"ebay, inc.",site operations,rover.ebay.com
0,18,18,9,9,9,5,9,18,0,0,...,0,0,0,0,0,0,0,0,0,0
1,15,0,0,0,0,0,0,0,10,10,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,32,0,0,0,0,0,0,0,18,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
5996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5997,2,2,0,0,0,0,3,6,0,0,...,0,0,0,0,0,0,0,0,0,0
5998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [126]:
files_test = os.listdir(test_path)

In [127]:
for file in files_test:
    path = os.path.join(test_path, file)
    analyze(path, False)

In [128]:
test_data = pd.read_csv(test_csv_path)

In [129]:
test_IP = test_data['file']

In [130]:
test_data = test_data.drop(['file'], axis=1)

In [131]:
test_data

,globalsign nv-sa,globalsign rsa ov ssl ca 2018,massachusetts,boston,"logmein, inc.",secure.logmeinrescue.com,globalsign root ca - r3,globalsign,globalsign organization validation ca - sha256 - g2,beijing,...,bigoterr3a.com,"block.io, inc.",block.io,it department,lct.avictc.com,vxcharts.com,bdns.at,"ebay, inc.",site operations,rover.ebay.com
0,0,0,0,0,0,0,0,12,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,0,0,0,0,0,0,0,2,2,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,4,10,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,12,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3998,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0


In [132]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

In [133]:
rfc.fit(train_data, train_label)

RandomForestClassifier()

In [134]:
white_count = 0
black_count = 0
results = rfc.predict(test_data)
count = 0
result_black_list = []
result_white_list = []
for result in results:
    IP = test_IP[count][:-5]
    if result == 0:
        white_count += 1
        result_white_list.append(IP + '\n')
    else:
        black_count += 1
        result_black_list.append(IP + '\n')
    count += 1
print(white_count, black_count)

2430 1570


1